In [1]:
!pip install langchain_community
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.1/420.1 kB 26.5 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
E

In [2]:
import pandas as pd
recipes_df = pd.read_csv("/kaggle/input/final-dataset/final_dataset.csv")

In [3]:
import sys
sys.path.append('/kaggle/input/final-sustainameal-package')
from sustainameal import SustainaMeal

#nutrients = ['calories [cal]', 'totalFat [g]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]']
nutrients = ['calories [cal]', 'totalFat [g]', 'saturatedFat [g]', 'cholesterol [mg]', 'sodium [mg]', 'dietaryFiber [g]', 'sugars [g]', 'protein [g]']

sm = SustainaMeal( recipes_df=recipes_df, 
                  nutrients=nutrients, 
                  transformer_name='davanstrien/autotrain-recipes-2451975973', 
                  load=True )

tokenizer_config.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [6]:
import pandas as pd
from tabulate import tabulate

acceptable_tags = ['appetizers', 'main-dish', 'side-dishes', 'drinks', 'beverages', 'fruit', 'desserts',
                   'breakfast', 'pasta-rice-and-grains', 'rice', 'pasta', 'pizza', 'breads', 'meat', 'fish',
                   'seafood', 'beef', 'chicken', 'vegetarian', 'gluten-free', 'crustacean-free', 'egg-free',
                   'fish-free', 'peanut-free', 'soy-free', 'lactose-free', 'nut-free', 'celery-free',
                   'sesame-free', 'sulfite-free', 'lupin-free', 'mollusk-free', 'mustard-free']

title = 'Good Times and Noodle Salad (Pasta Salad)'
#print("Title: ", title)
specific_tag = 'pasta'
similar_recipes = sm.find_similar_recipes(title, 10, acceptable_tags=specific_tag, match_all_tags=True, check_sustainability=False)

def get_tags_from_title(df, titles):
    if isinstance(titles, str):
        titles = [titles]
    
    result = df[df['title'].isin(titles)][['title', 'tags']]
    
    if result.empty:
        return None
    
    return {row['title']: [tag.strip().lower().replace("'", "").replace('"', "").replace(" ", "") for tag in row['tags'].split(', ')] if isinstance(row['tags'], str) else [] for _, row in result.iterrows()}

def extract_recipe_titles(recipe_list):
    return [recipe[1] for recipe in recipe_list]

def calculate_tag_similarity(df, target_title, similar_titles, specific_tag=None):
    target_tags = get_tags_from_title(df, target_title)
    if not target_tags or target_title not in target_tags:
        return None
    
    target_tags_set = set(tag.strip().lower().replace("'", "").replace('"', "").replace("[", "").replace("]", "").replace("\'", "").replace(" ", "") for tag in target_tags[target_title])
    similarities = []
    
    for title in similar_titles:
        recipe_tags = get_tags_from_title(df, title)
        if not recipe_tags or title not in recipe_tags:
            continue
        
        tag_set = set(tag.strip().lower().replace("'", "").replace('"', "").replace("[", "").replace("]", "").replace("\'", "").replace(" ", "") for tag in recipe_tags[title])
        common_tags = target_tags_set.intersection(tag_set)
        similarity_percentage = (len(common_tags) / len(target_tags_set)) * 100 if target_tags_set else 0
        
        # Debugging: Print tag sets for verification
        has_specific_tag = specific_tag.strip().lower().replace("'", "").replace('"', "").replace("[", "").replace("]", "").replace("\'", "").replace(" ", "") in tag_set if specific_tag else None
        
        similarities.append([title, similarity_percentage, has_specific_tag, ', '.join(common_tags) if common_tags else 'None'])
    
    return pd.DataFrame(similarities, columns=["title", "commonTagsPercentage", f"Contains '{specific_tag}'", "common_tags"])

titles = extract_recipe_titles(similar_recipes)
print("TITLES: ", titles)
matching_tags_scores = calculate_tag_similarity(recipes_df, title, titles, specific_tag)

if not matching_tags_scores.empty:
    print("Matching Recipes:")
    pd.set_option('display.max_colwidth', None)
    print(matching_tags_scores)
else:
    print("No common tags found.")
pd.set_option('display.max_colwidth', None)
print("Similar recipes\n: ", similar_recipes)
for recipe in similar_recipes:
    tags = recipes_df.loc[recipes_df['title'] == recipe[1], 'tags']
    print('tags:', tags)

print("----------------------------------")


ordered_recipes = pd.DataFrame(sm.order_recipe_by_healthiness(), columns=["title", "similarity_score", "who_score", "healthiness_increment"])
print("Ordered Recipes by Healthiness:")
print(tabulate(ordered_recipes, headers='keys', tablefmt='grid'))

ordered_recipes_sus = pd.DataFrame(sm.order_recipe_by_sustainability(), columns=["title", "similarity_score", "sustainability_score", "who_score", "sustainability_increment"])
print("Ordered Recipes by Sustainability:")
print(tabulate(ordered_recipes_sus, headers='keys', tablefmt='grid'))

order_recipes_final = pd.DataFrame(sm.order_recipe_by_sustainameal(), columns=["title", "similarity_score", "who_score", "healthiness_increment", "sustainability_score", "sustainability_increment", "sustainameal_score", "sustainameal_score_increment"])
print("Ordered Recipes by Sustainameal:")
print(tabulate(order_recipes_final.rename(columns={'similarity_score': 's_score', 'healthiness_increment': 'h_increment', 'sustainability_score': 'sus_score', 'sustainability_increment': 'sus_increment'}), headers='keys', tablefmt='grid', showindex=False, maxcolwidths=[15, 15]))


Processing Titles embeddings: 100%|██████████| 1/1 [00:00<00:00, 90.98batch/s]


Number of filtered recipes: 3558
TITLES:  ['Perfect Greek Orzo Salad', 'Lemon Orzo With Mint', 'Pasta Salad, Fabulous No Mayo!', "Barbara's Chinese Noodle Salad", 'Italian Vegetable Pasta Salad', 'Sesame Noodles With Napa Cabbage', 'Greek Pasta Shells Salad', 'Asian Vermicelli', 'Noodles With Stir-Fried Tofu and Broccoli', 'Kung Fu Pasta Salad']
Matching Recipes:
                                       title  commonTagsPercentage  \
0                   Perfect Greek Orzo Salad                87.500   
1                       Lemon Orzo With Mint                68.750   
2             Pasta Salad, Fabulous No Mayo!                93.750   
3             Barbara's Chinese Noodle Salad                62.500   
4              Italian Vegetable Pasta Salad                81.250   
5           Sesame Noodles With Napa Cabbage                65.625   
6                   Greek Pasta Shells Salad                78.125   
7                           Asian Vermicelli                78.125   
8  N